In [2]:
# importing libraries
import numpy as np
import pandas as pd
from pathlib import Path
import hvplot.pandas
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import time
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import MeanSquaredError
from pandas.tseries.offsets import DateOffset
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from bokeh.models.formatters import NumeralTickFormatter
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.linear_model import LogisticRegression


%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


2023-07-26 17:00:35.409523: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
all_values = pd.read_csv(Path("Resources/all_values.csv"))
all_values['date'] = pd.to_datetime(all_values['date'], format='%Y-%m')
all_values.set_index('date', inplace=True)

In [10]:
scaler_all = StandardScaler()
all_scaler = scaler_all.fit(all_values)
all_values_scaled = all_scaler.transform(all_values)

all_values_scaled_df = pd.DataFrame(all_values_scaled, columns=all_values.columns, index=all_values.index)
all_values_scaled_df.head()

,Composite,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,,
2008-06-01,-1.131020,-0.672479,-0.469992,-0.992346,-0.858730,2.172972,-1.310833,-1.693840,-1.168618,-1.911253,0.170914,-0.692024,-1.836365,-1.370649,-1.488305,-1.180402,-1.390596
2008-07-01,-1.137164,-0.645519,-0.552576,-1.105315,-0.943586,2.172972,-1.264221,-1.641446,-1.088744,-1.884173,0.677639,-0.692024,-1.823707,-1.386239,-1.493945,-1.150188,-1.380045
2008-08-01,-1.142285,-0.612804,-0.453057,-0.986607,-0.873159,2.172972,-1.264221,-1.589052,-1.048807,-1.884173,0.360936,-0.803609,-1.709782,-1.339471,-1.499586,-1.170331,-1.358941
2008-09-01,-1.146893,-0.728685,-0.496322,-0.872477,-0.765473,2.172972,-1.254898,-1.510461,-1.104719,-1.762314,0.170914,-0.810583,-1.722441,-1.355060,-1.471383,-1.170331,-1.358941
2008-10-01,-1.163279,-0.891076,-1.336555,-1.199396,-1.016262,1.332881,-1.329479,-1.477714,-1.096731,-1.789394,-0.652515,-1.089543,-1.709782,-1.386239,-1.471383,-1.281118,-1.380045


In [74]:
all_values_scaled_df.tail()

,Composite,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,,
2022-12-01,1.712419,0.000103,1.137179,1.463593,1.599745,3.573123,2.362250,2.464931,2.465659,1.934097,1.501068,2.418384,2.429464,2.105790,1.754984,2.334585,2.249772
2023-01-01,1.635611,0.402533,1.227441,1.681039,1.842122,3.853153,2.399540,2.576268,2.481634,1.866397,1.311046,2.390488,2.518071,2.105790,1.828311,2.344657,2.270875
2023-02-01,1.604375,0.334344,1.224208,1.905592,2.050030,3.853153,2.418185,2.628662,2.505596,2.015337,1.817771,2.299826,2.606679,2.059022,1.879075,2.364800,2.323634
2023-03-01,1.631514,0.105736,0.965558,1.744085,1.965177,3.853153,2.455476,2.681056,2.553521,2.001797,2.007793,2.271930,2.707945,2.152559,1.952402,2.395015,2.376393
2023-04-01,1.728292,-0.003839,0.959360,1.828210,2.176525,3.853153,2.539379,2.739999,2.633395,2.028877,1.881112,2.397462,2.796553,2.246095,1.974964,2.485659,2.429152


In [11]:
y = all_values_scaled_df["Composite"]

In [12]:
X = all_values_scaled_df.drop(columns = "Composite")

In [13]:
y

date
2008-06-01   -1.131020
2008-07-01   -1.137164
2008-08-01   -1.142285
2008-09-01   -1.146893
2008-10-01   -1.163279
                ...   
2022-12-01    1.712419
2023-01-01    1.635611
2023-02-01    1.604375
2023-03-01    1.631514
2023-04-01    1.728292
Name: Composite, Length: 179, dtype: float64

In [14]:
X

,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,
2008-06-01,-0.672479,-0.469992,-0.992346,-0.858730,2.172972,-1.310833,-1.693840,-1.168618,-1.911253,0.170914,-0.692024,-1.836365,-1.370649,-1.488305,-1.180402,-1.390596
2008-07-01,-0.645519,-0.552576,-1.105315,-0.943586,2.172972,-1.264221,-1.641446,-1.088744,-1.884173,0.677639,-0.692024,-1.823707,-1.386239,-1.493945,-1.150188,-1.380045
2008-08-01,-0.612804,-0.453057,-0.986607,-0.873159,2.172972,-1.264221,-1.589052,-1.048807,-1.884173,0.360936,-0.803609,-1.709782,-1.339471,-1.499586,-1.170331,-1.358941
2008-09-01,-0.728685,-0.496322,-0.872477,-0.765473,2.172972,-1.254898,-1.510461,-1.104719,-1.762314,0.170914,-0.810583,-1.722441,-1.355060,-1.471383,-1.170331,-1.358941
2008-10-01,-0.891076,-1.336555,-1.199396,-1.016262,1.332881,-1.329479,-1.477714,-1.096731,-1.789394,-0.652515,-1.089543,-1.709782,-1.386239,-1.471383,-1.281118,-1.380045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-01,0.000103,1.137179,1.463593,1.599745,3.573123,2.362250,2.464931,2.465659,1.934097,1.501068,2.418384,2.429464,2.105790,1.754984,2.334585,2.249772
2023-01-01,0.402533,1.227441,1.681039,1.842122,3.853153,2.399540,2.576268,2.481634,1.866397,1.311046,2.390488,2.518071,2.105790,1.828311,2.344657,2.270875
2023-02-01,0.334344,1.224208,1.905592,2.050030,3.853153,2.418185,2.628662,2.505596,2.015337,1.817771,2.299826,2.606679,2.059022,1.879075,2.364800,2.323634


In [ ]:
all_values_scaled_df.hvplot(
    xlabel='Date', 
    ylabel='Scaled Value', 
    title='Scaled Values',
    height=800,
    width=2000
)

In [15]:
training_begin = X.index.min()
print(training_begin)
training_end = X.index.min() + DateOffset(months=125)
print(training_end)

2008-06-01 00:00:00
2018-11-01 00:00:00


In [27]:
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]
X_train.shape

(126, 16)

In [28]:
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]
X_test.shape

(54, 16)

In [47]:
# Split the data into training and test sets
# Feature elimination using SelectKBest based on ANOVA F-value
k_best = SelectKBest(score_func=f_classif, k=3)
X_train_kbest = k_best.fit_transform(X_train, y_train)
X_test_kbest = k_best.transform(X_test)


In [69]:
X_train_kbest.shape

(126, 3)

In [70]:
y_train.shape

(126,)

In [49]:
print("SelectKBest:", X_train_kbest.shape, X_test_kbest.shape)

SelectKBest: (126, 3) (54, 3)


In [50]:
selected_feature_names = X.columns[k_best.get_support()]

In [51]:
print(selected_feature_names)

Index(['XHB', 'Food', 'Household operations, furnishings and equipment'], dtype='object')


In [52]:
X_selected = X[selected_feature_names]

In [65]:
y_train

date
2008-06-01   -1.131020
2008-07-01   -1.137164
2008-08-01   -1.142285
2008-09-01   -1.146893
2008-10-01   -1.163279
                ...   
2018-07-01    0.341137
2018-08-01    0.353426
2018-09-01    0.363155
2018-10-01    0.373396
2018-11-01    0.363667
Name: Composite, Length: 126, dtype: float64

In [66]:
X_selected

,XHB,Food,"Household operations, furnishings and equipment"
date,,,
2008-06-01,-0.992346,-1.693840,-1.911253
2008-07-01,-1.105315,-1.641446,-1.884173
2008-08-01,-0.986607,-1.589052,-1.884173
2008-09-01,-0.872477,-1.510461,-1.762314
2008-10-01,-1.199396,-1.477714,-1.789394
...,...,...,...
2022-12-01,1.463593,2.464931,1.934097
2023-01-01,1.681039,2.576268,1.866397
2023-02-01,1.905592,2.628662,2.015337


In [60]:
# Neural network parameters
n_input_feats = len(X_selected.columns)

n_outputs = 1

hidden_nodes_layer1 = 128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 32
# hidden_nodes_layer4 = 4
print(f"""
Neural Network Parameters:
--------------------------
Input Features: {n_input_feats}
Layers:         3
Layer1 Nodes:   {hidden_nodes_layer1}
Layer2 Nodes:   {hidden_nodes_layer2}
Layer3 Nodes:   {hidden_nodes_layer3}
Output Nodes:   {n_outputs}
""")


Neural Network Parameters:
--------------------------
Input Features: 3
Layers:         3
Layer1 Nodes:   128
Layer2 Nodes:   64
Layer3 Nodes:   32
Output Nodes:   1



In [61]:
# define model
nn = Sequential()
# Add the first hidden layer
nn.add(Dense(
    units=hidden_nodes_layer1,
    input_dim = n_input_feats,
    activation='relu',
    name='hidden1'
))
# Add the second hidden layer
nn.add(Dense(
    units=hidden_nodes_layer2,
    activation='relu',
    name='hidden2'
))
# Add the third hidden layer
nn.add(Dense(
    units=hidden_nodes_layer3,
    activation='relu',
    name='hidden3'
))
# Add output
nn.add(Dense(
    n_outputs,
    activation='linear',
    name='output'
))

In [62]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden1 (Dense)             (None, 128)               512       
                                                                 
 hidden2 (Dense)             (None, 64)                8256      
                                                                 
 hidden3 (Dense)             (None, 32)                2080      
                                                                 
 output (Dense)              (None, 1)                 33        
                                                                 
Total params: 10,881
Trainable params: 10,881
Non-trainable params: 0
_________________________________________________________________


In [63]:
# Compile the Sequential model
nn.compile(loss='mean_squared_error', optimizer='adam', metrics=[MeanSquaredError()])

In [64]:
# Fit the model 
model = nn.fit(
    X_selected,
    y_train,
    batch_size=10,
    validation_split=0.2,
    epochs=100
)

ValueError: Data cardinality is ambiguous:
  x sizes: 143
  y sizes: 126
Make sure all arrays contain the same number of samples.

In [ ]:
# R2 score for training
train_pred = nn.predict(X_train)
training_r2 = r2_score(y_train.values, train_pred)
training_r2

In [ ]:
train_predictions_df = X_train.copy()
train_predictions_df['Train Predictions'] = train_pred
cols = train_predictions_df.columns.tolist()
cols = [cols[-1]] + cols[:-1]
train_predictions_df = train_predictions_df[cols]

In [ ]:
train_pred_descaled_df = pd.DataFrame(all_scaler.inverse_transform(train_predictions_df), columns=train_predictions_df.columns, index=train_predictions_df.index)
#train_pred_descaled_df['Actual'] = all_values['Composite'][:126].values
train_pred_descaled_df

In [ ]:
# Create a DataFrame with the history dictionary
model_df = pd.DataFrame(model.history, index=range(1, len(model.history["loss"]) + 1))

In [ ]:
model_df[['val_loss', 'mean_squared_error']].hvplot(
height=800,
    width=2000
)

In [ ]:
predictions = nn.predict(X_test)

In [ ]:
# R2 score from test
test_r2 = r2_score(y_test.values, predictions)
test_r2

In [ ]:
predictions_df = X_test.copy()
predictions_df['Test Predictions'] = predictions
cols = predictions_df.columns.tolist()
# Move the last column to the first position
cols = [cols[-1]] + cols[:-1]
# Reindex the DataFrame with the new column order
predictions_df = predictions_df[cols]
predictions_df.head()

In [ ]:
# Inverse transform for real values
pred_descaled_df = pd.DataFrame(all_scaler.inverse_transform(predictions_df), columns=predictions_df.columns, index=predictions_df.index)
pred_descaled_df

In [ ]:
total_chart = pd.concat([train_pred_descaled_df, pred_descaled_df],axis=1) 
total_chart['Actual'] = all_values['Composite'].values

In [ ]:
total_chart[['Actual', 'Train Predictions', 'Test Predictions']].hvplot(
    xlabel='Date', 
    ylabel='Price', 
    title='Actual and Predicted Index',
    height=800,
    width=2000
).opts(
    yformatter=NumeralTickFormatter(format="0,0")
)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def weighted_absolute_percentage_error(y_true, y_pred):
    return 100 / len(y_true) * np.sum(np.abs((y_true - y_pred) / y_true))

def weighted_mean_absolute_percentage_error(y_true, y_pred):
    return 100 / sum(y_true) * np.sum(np.abs(y_true - y_pred))

In [ ]:
# Generate predictions
y_pred = nn.predict(X_test)

# Calculate R-Squared
r2 = r2_score(y_test, y_pred)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)

# Calculate Mean Absolute Percentage Error
mape = mean_absolute_percentage_error(y_test, y_pred)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)

# Calculate Normalized Root Mean Squared Error
nrmse = rmse / (np.max(y_test) - np.min(y_test))

# Calculate Weighted Absolute Percentage Error
wape = weighted_absolute_percentage_error(y_test, np.squeeze(y_pred))

# Calculate Weighted Mean Absolute Percentage Error
wmape = weighted_mean_absolute_percentage_error(y_test, np.squeeze(y_pred))

In [ ]:
print(f"R-Squared: {r2}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Absolute Percentage Error: {mape}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Normalized Root Mean Squared Error: {nrmse}")
print(f"Weighted Absolute Percentage Error: {wape}")
print(f"Weighted Mean Absolute Percentage Error: {wmape}")

In [ ]:
nn_json = nn.to_json()

file_path = ("Models/model1.json")
with open(file_path, "w") as json_file:
    json_file.write(nn_json)

In [ ]:
file_path = "Models/model1.h5"
nn.save_weights(file_path)